### Step 1: Import Packages

- `langchain` → agent framework
- `langchain-openai` → connects to GPT-4
- `gradio` → web UI
- `pypdf`, `pdf2image`, `Pillow` → PDF → text & images
- `chromadb`, `tiktoken`, `transformers` → optional advanced features (embeddings, token counting)

In [ ]:
# ─── Cell 1: Install Packages  ─────────────────────────────────
!pip install --upgrade \
  "langchain==0.1.16" \
  "langchain-core==0.1.45" \
  "langchain-openai==0.1.3" \
  "openai>=1.0.0" \
  "gradio>=4.0.0" \
  "chromadb<0.5.0" \
  "sentence-transformers<3.0.0" \
  "pypdf<4.0.0" \
  "tiktoken" \
  "transformers<5.0.0" \
  "python-dotenv<1.1.0" \
  "pdf2image<1.17.0" \
  "pillow<11.0.0"

In [ ]:
# ─── Cell 2: Imports & Setup  ─────────────────────────────────
import os, json, io, base64
from dotenv import load_dotenv
from typing import List
from PIL import Image
from pdf2image import convert_from_path
import gradio as gr

from openai import OpenAI as OpenAIClient
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

from pypdf import PdfReader

from google.colab import userdata

### Step 2: Define the Tools

```
User query → Agent (LLM)
               ↓
          reads tools' docstrings
               ↓
          picks a tool → calls function → returns result
               ↓
          formats answer → back to user

In [ ]:
# ─── Cell 3: Tool 1 (parse_resume) ──────────────────────────────────────────
@tool
def parse_resume(pdf_path: str, job_spec: str) -> dict:
    """
    Extract the raw text of a résumé PDF and carry along the job_spec.

    Args:
      pdf_path: Path to the uploaded PDF file
      job_spec: Target job role/description (e.g., "Software Engineer")

    Returns:
      dict with keys 'text' (str) and 'job_spec' (str)
    """


In [ ]:
# ─── Cell 4: Tool 2 (enhance_resume) ──────────────────────────────────────────
@tool
def enhance_resume(text: str, job_spec: str) -> str:
    """
    Rewrite a résumé text for better clarity, impact, and relevance to the job_spec.

    Args:
      text: Raw résumé text (output from parse_resume)
      job_spec: Target role (e.g., "Product Manager")

    Returns:
      Enhanced résumé text (str)
    """


In [ ]:
# ─── Cell 5: Tool 3 (vision_style_analyzer) ──────────────────────────────────────────
@tool
def vision_style_analyzer(pdf_path: str) -> dict:
    """
    Analyze the first page of the resume using GPT-4 Vision.

    Returns a dict with:
      - style_score (int): design score 1 (poor) to 10 (excellent)
      - template_type (str): e.g. 'classic', 'modern', 'creative'
      - positive_points (List[str]): what works visually
      - suggestions (List[str]): concrete layout/design improvements
    """
    # 1. Convert PDF to Image
    pages = convert_from_path(pdf_path, dpi=250, first_page=1, last_page=1)
    img = pages[0].convert("RGB")

    # 2. Resize if Needed (Vision Model Limit)
    w, h = img.size
    if w > 2048:
        img = img.resize((2048, int(2048 * h / w)), Image.LANCZOS)

    # 3. Compress to JPEG
    buf = io.BytesIO()
    img.save(buf, format="JPEG", quality=75)
    buf.seek(0)

    # 4. Encode as Base64 Data URI
    b64_img = base64.b64encode(buf.getvalue()).decode("ascii")
    image_url = f"data:image/jpeg;base64,{b64_img}"

    # 5. Call GPT-4 Vision using the modern OpenAI v1.x+ client
    client = OpenAIClient()
    prompt_text = (
      "You are a senior hiring manager at a top tech firm with a background in graphic design. "
      "Your critique must be brutally honest and focused on what will get a candidate noticed or rejected based on visual presentation alone. "
      "Analyze the attached résumé image and provide a detailed critique. "
      "Do not give generic advice like 'add more white space' unless the document is genuinely cramped; instead, point to specific sections that need it. "
      "Your response MUST be a single, raw JSON object with the following keys:\n"
      "  'style_score': An integer from 1-10 based on its immediate professional impact.\n"
      "  'template_type': One of 'classic', 'modern', or 'creative'.\n"
      "  'positive_points': An array of strings detailing what is visually effective (e.g., 'Good use of columns for readability').\n"
      "  'improvement_suggestions': An array of actionable, specific strings for visual improvement (e.g., 'The font size for section headers is inconsistent; make all headers 14pt').\n\n"
      "Provide ONLY the raw JSON object and nothing else."
    )

    # Build the Messages Array
    messages = [
        {
            "role": _, # Fill in the blank
            "content": "You are an expert design critic providing feedback as a JSON object."
        },
        {
            "role": _", # Fill in the blank
            "content": [
                # Part 1:
                {"type": "text", "text": _}, # Fill in the blank
                # Part 2:
                {
                    "type": "image_url",
                    "image_url": {
                        "url": _, # Fill in the blank
                        "detail": "_" # Fill in the blank
                    }
                }
            ]
        }
    ]

    # Make the API Call

    # 6. Parse and return


### Step 3: Define the Instruction


In [ ]:
# ─── Cell 6: Define the Instruction/System Prompt  ────────
SYSTEM_MESSAGE = SystemMessage(
    content="""
    You are the

      1. parse_resume(pdf_path: str, job_spec: str) →
        •
      2. enhance_resume(text: str, job_spec: str) →
        •
      3. vision_style_analyzer(pdf_path: str) →
        •

    Workflow rules:

"""
)

#### Step 4: Initialise the Model


In [ ]:
# ─── Cell 7: Initialise the Model ──────────────────────────────────────────

# 1. Set OpenAI API key from Colab Secrets

# 2. Assemble the tools list

# 3. Initialize the LLM

# 4. Build the prompt template

#### Step 5: Initialise the AI Agent


In [ ]:
# ─── Cell 8: Initialise the AI Agent ──────────────────────────────────────────

# 1. Create the agent using the modern 'create_openai_tools_agent' function

# 2. Create the AgentExecutor, which will run the agent and tools in a loop


#### Step 5: Initialise the Interface

**What this step does:**

-   Creates a Gradio web interface with file upload, text input, and output display
-   Wires two buttons ("Enhance Résumé" and "Critique Style") to the agent executor
-   Launches a shareable web app for interacting with the AI agent

**Key definitions:**

-   **Gradio (`gr.Blocks`)**: A Python library for building ML/AI web UIs with minimal code
-   **`gr.File`**: File upload component---returns a file object with `.name` (path on disk)
-   **`gr.Textbox`**: Text input/output component
-   **`gr.Button.click()`**: Connects a button to a Python function; when clicked, runs the function with the specified inputs and displays the result in outputs
-   **`.queue().launch(share=True)`**: Starts the web server; `share=True` creates a public URL (valid for 72 hours)

**Why Gradio for this project:**

-   **Rapid prototyping**: 20 lines of code to go from working agent to working demo
-   **No frontend knowledge needed**: Pure Python---no HTML, CSS, or JavaScript
-   **Shareable**: `share=True` generates a public link you can send to colleagues or clients
-   **Perfect for agents**: Gradio's simplicity lets us focus on the agent logic, not UI design

**How the interface works:**

1.  **User uploads a PDF** → Gradio saves it to `/tmp/gradio/...` and passes the file object to your function
2.  **User enters a job spec** (e.g., "Senior Data Scientist") → stored in the `job_spec` textbox
3.  **User clicks "Enhance Résumé"** → triggers the lambda function:

```
   lambda pdf, spec: run_agent(
       f"Please enhance the résumé at {pdf.name} for a {spec} position."
   )
```

  -   `pdf.name` is the file path (e.g., `/tmp/gradio/abc123/resume.pdf`)
  -   The lambda constructs a natural language query and passes it to `run_agent()`

4.  **`run_agent()` calls the agent executor** → agent parses → enhances → returns result
5.  **Result displayed in the output textbox**

**The two workflows:**
| Button | Query Template | Agent's Action |
| --- | --- | --- |
| **Enhance Résumé** | `"Please enhance the résumé at {path} for a {spec} position."` | Calls `parse_resume` → `enhance_resume` → returns improved text |
| **Critique Style** | `"Please critique the visual design and template of the résumé at {path}."` | Calls `vision_style_analyzer` → returns JSON with score, type, suggestions |

In [ ]:
# ─── Cell 9: Initialise the Interface ──────────────────────────────────────────
with gr.Blocks() as demo:
    gr.Markdown("## AI‐Powered Résumé Enricher + Style Critic")

    with gr.Row():
        # This tells Gradio to accept any file and lets our Python code do the real validation.
        resume_file = gr.File(label="Upload Résumé (PDF)")
        job_spec    = gr.Textbox(label="Job Specification",
                                 placeholder="e.g. Senior Software Engineer")

    btn_enhance = gr.Button("Enhance Résumé")
    btn_style   = gr.Button("Critique Style")
    output      = gr.Textbox(label="Result", lines=20)

    # Function to handle the agent invocation and extract the output
    def run_agent(query):
        response = agent_executor.invoke({"input": query})
        return response['output']

    # Launch enhancement flow
    btn_enhance.click(
        fn=lambda pdf, spec: run_agent(
            f"Please enhance the résumé at {pdf.name} for a {spec} position."
        ),
        inputs=[resume_file, job_spec],
        outputs=output,
    )

    # Launch style-critique flow
    btn_style.click(
        fn=lambda pdf: run_agent(
            f"Please critique the visual design and template of the résumé at {pdf.name}."
        ),
        inputs=[resume_file],
        outputs=output,
    )

    demo.queue().launch(share=True, debug=True)